# Introduction

We are collecting statistics for following units
* level: 6 -> gmina
* kind: 1, 4 -> 'miasto powiatowe' or city in 'gmina miejsko-wiejska'
* in case a city turned from 'wies' to city in 'gmina miejsko-wiejska' we don't take under consideration statistics befor a 'city' state, it is not that much relevant and additionally it would be most probably treated as a small city - we are only interested in medium size cities
* when a city turned from city in 'gmina miejsco-wiejska' to 'miasto powiatowe' kind 1 -> 4 we combine both statistics to keep continouity and remove double stats, the best if mapping is dynamically generated as we may have plenty of cases like these
* in case a village turned into a city in current year we won't have statistics for that unit -> we should remove this unit from our research, the same should happen when statistics for a unit do not cover full 5 years, ex. income is only collected for 2 years in 5 years period 
* in case a city turned into a new administative unit: case of Warszawa or Walbrzych, we need to filter that out manually

In order to identify city status we have to group statistics in various categories
* population ...

We have 2 main periods for research - before 2003 and after -> unemployment rate
In order to take under consideration a city to our research it must have values for each parameter

for period 2003 earlier we use different mapping of points. If we divide to more groups we could have much more accurate mapping when unemployment rate is not collected.

!!! Very important thing. If for any city we don't have statistics collected in each parameter we don't take under consideration this city in our research for given period

- überdurchschnittlich wachsend: 19 bis 24 Punkte
- wachsend: 14 bis 18 Punkte
- keine eindeutige Entwicklungsrichtung: 11 bis 13 Punkte
- schrumpfend: 6 bis 10 Punkte
- überdurchschnittlich schrumpfend: 0 bis 5 Punkte. 

TODO 
Another data cleaninig procedure is to fill gaps for employed and unemlpoyed based on past stats for 'gmina' level 
and taking under consideration population, working age ratio

For unemployed we have to do following
* reversed quantile as the lower unemployed rate the better, 
* collect mean from 2 years to avoid high fluctuations
* divide cities into 3 groups not to penalized cities with already low unemployed rate - but based on what?? tbd

TODO 
We may need to update unify unit ids just for gminas as they could have been moved etc. 
We will use gminas for a data input for employment and unemployed parameters

TODO 
properly define column types that they don't take too much space when calculating

Loooks like we will have to do a unifiquation as well for 'gmina' level. as we may have the same problems whenver a gmina moved to a new powiat to keep continuity.

TODO 
When we fill gaps in values for specific units based on parent value and ratio of people in working age to total people in working age in whole parent unit that ratio is much smaller than in reality, that means cities have extra bonus based on something and not only based on working age. If we would use only population that ratio is even smaller

TODO 
Use geometric mean instead of regular mean for all parameters beside migration
Migration is still done as a regular mean

TODO 
for unemployment divide cities based on 
- city type, 
- unemployment level - but this would have to be averaged for given period and use predefined values 0.1 - 1.0 etc
- divide them by the avg of the employment rate -0.5 - +0.5 
- use normal distribution 

TODO 
Readjustment score based on pre-defined values from Milbert Methodology

TODO
* 1 - divide unemployment based on specific parameter - so far it's only based on quintile
* 2 - extrapolate, approximate data for gaps based on trends of parent unit and not based on ratio of working age in unit vs parent municipality
* 3 - polish data with 2 year intervals



# Init

## Imports

In [1]:
import pandas as pd
import numpy as np
import re
import time

## Common

In [2]:
# ************************************************** 
# creates a csv file name with a time linked as a suffix
# ************************************************** 
def csv_name(path:str, name:str, suffix='') -> str:
    now = '_' + str(int(time.time()))
    return path + '/' + name + suffix + now + '.csv'

## Units

In [3]:
# ************************************************** 
# logical units which combine units virtually
# ************************************************** 
logical_units = [
    '011212210001', # Miejska strefa usług publicznych
    '030210216001', # Miejska strefa usług publicznych
    '040416710001', # Miejska strefa usług publicznych
    '071412831981', # Związek gmin dzielnic Warszawy do 2001
    '071412831991', # GMINY-DZIELNICY WARSZAWY NIE USTALONO do 2001
    '020811409102', # Zielona Góra - gmina combined with city
    '071412912031'  # Wesoła - city combined with Warszawa and became it's district
    ]

# ************************************************** 
# warsaw districts do not bring any added value to analysis, 
# and they are obsolete after 2001 -> new administration unit is created
# ************************************************** 
warsaw_districts = [
    '071412831011', # Warszawa - Bemowo do 2001
    '071412831021', # Warszawa - Białołęka do 2001
    '071412831031', # Warszawa - Bielany do 2001
    '071412831041', # Warszawa - Centrum do 2001
    '071412831121', # Warszawa - Rembertów do 2001
    '071412831131', # Warszawa - Targówek do 2001
    '071412831141', # Warszawa - Ursus do 2001
    '071412831151', # Warszawa - Ursynów do 2001
    '071412831161', # Warszawa - Wawer do 2001
    '071412831171', # Warszawa - Wilanów do 2001
    '071412831181'  # Warszawa - Włochy do 2001
    ]

# ************************************************** 
# untis with keys should be removed, while data enries shuld be unified to value
# ************************************************** 
merge_dict = { 
    '011216911021': '011216911024', # Szczawnica
    '030210302031': '030210302034', # Pieszyce
    '042214011021': '042214011024', # Jastarnia
    '042214011041': '042214011044', # Władysławowo
    '042214213011': '042214213014', # Czarna Woda
    '071412912021': '071412912151', # Sulejówek
    '060611211084': '060611211021', # Stoczek Łukowski
    '071412831001': '071412865011', # M.st.Warszawa
    '030210363011': '030210365011', # Wałbrzych
    '030210321091': '030210365011', # Wałbrzych
    '011212312021': '012415001081', # Sławków
    '020811404094': '020811412014', # Sława
    '020811404104': '020811412024', # Szlichtyngowa
    '020811404114': '020811412034', # Wschowa
    '023216604014': '023216418014', # Dobra
    '023216605064': '023216418044', # Resko
    '023216614074': '023216418024', # Łobez
    '023216614124': '023216418054', # Węgorzyno
    '042214209024': '042214216014', # Dzierzgoń
    '042214209114': '042214216054', # Sztum
    '042815506094': '042815519034', # Węgorzewo
    '042815513024': '042815518034', # Gołdap
    '051011506011': '051011521011', # Brzeziny
    '061813301044': '061813321034'  # Lesko
    }

## Cities

In [4]:
# ************************************************** 
# city related functions used mostly in statistics
# **************************************************
small_city = { 'id': 'small', 'type': 'small', 'min_size': 0, 'max_size': 20000}
medium_city = { 'id': 'medium', 'type': 'medium', 'min_size': 20000, 'max_size': 100000}
large_city = { 'id': 'large', 'type': 'large', 'min_size': 100000, 'max_size': 100000000}
size_margin = 0.1

def city_type(size: int) -> int:
    medium_min = medium_city['min_size'] - size_margin * medium_city['min_size']
    medium_max = medium_city['max_size'] + size_margin * medium_city['max_size']
    if size >= medium_min:
        if size <= medium_max:
            return medium_city['id']
        else:
            return large_city['id']
    else:
        return small_city['id']

# ************************************************** 
# city shrinking status
#   0-6 -> significantly shrinking
#  7-10 -> shrinking
# 11-13 -> stablisation
# 14-18 -> growing
# 19-24 -> significantly growing
# **************************************************
def city_status(score: int) -> str:
    if score <= 6: return 'E'
    elif score <= 10: return 'D'
    elif score <= 13: return 'C'
    elif score <= 18: return 'B'
    elif score <= 24: return 'A'
    else: return 'F'

def city_name(name: str) -> str:
    res = name
    res = re.sub(' - miasto.*', '', res)
    res = re.sub(' od.*', '', res)
    res = re.sub(' do.*', '', res)

    return res

def teryt_id(unit_id: str) -> str:
    return unit_id.str[2:4] + unit_id.str[7:12]

# ************************************************** 
# adds a parent id from specific level above to all 
# 0 -> poland; 1 -> macro-region; 2 -> voivodeship; 3 -> region
# 4 -> sub-region; 5 -> county; 6 -> community; 7 -> statistical unit
# **************************************************
def add_master_id(df, units_df, level):
    df['master_id'] = df['parentId']

    while True:
        df = pd.merge(df, units_df[['id', 'level', 'parentId']], left_on='master_id', right_on='id', suffixes=('', '_drop'))
        df['master_id'] = np.where(df['level'] == level, df['master_id'], df['parentId_drop'])
        df.drop([col for col in df.columns if 'drop' in col], axis=1, inplace=True)

        if df.loc[df.level != level, 'level'].count() == 0:
            df.drop(['level'], axis=1, inplace=True)
            break

        df.drop(['level'], axis=1, inplace=True)

    return df

    

## Statistics

In [5]:
# ************************************************** 
# statistic periods
# **************************************************
sort_order = ['id', 'year']
periods = [1996, 2001, 2006, 2011, 2016]
# periods = [2006, 2011, 2016]
period_length = 5

# ************************************************** 
# quantile distribution
# **************************************************
quantile_distribution = [.2, .4, .6, .8]

# v is value, and q is list of quantiles ordered 
def quantile_score(v: float, q: list[float], reversed=False) -> int:
    res = len(q)

    for i in range(len(q)):
        if v < q[i]:
            res = i
            break

    if reversed:
        res = len(q) - res

    return res

# ************************************************** 
# unify unit id to a single one
# **************************************************
def unify_id(id: str) -> str:
    if id in merge_dict:
        return merge_dict[id]
    
    return id

# ************************************************** 
# counts parameter ratio per 1000 population
# **************************************************
def param_ratio_per_1000(population, param) -> float:
    ratio = 0.001 * population
    result = param / ratio

    return result

# ************************************************** 
# counts ratio between two parameters
# **************************************************
def param_ratio(param1, param2) -> float:
    return param1 / param2



## DataFrames

In [6]:
# ************************************************** 
# loads a main df from the csv file and renames a value column
# **************************************************
def init_main_df(path, var_id, param):
    df = pd.read_csv(path, dtype={'unitId': str})
    df = df[df['varId'] == var_id].copy()
    df['unitId'] = df.apply(lambda row: unify_id(row['unitId']), axis=1)
    df.rename(columns={'val': param, 'unitId': 'id'}, inplace=True)
    df.drop(['varId'], axis=1, inplace=True)
    df.sort_values(sort_order, inplace=True)
    df.reset_index(drop=True, inplace=True)

    return df

# ************************************************** 
# craetes a step df which is the merge between cities - left 
# and main - right; which is specific df for given step
# **************************************************
def init_step_df(cities_df, main_df):
    df = pd.merge(cities_df, main_df, on='id')
    df.sort_values(sort_order, inplace=True)
    df.reset_index(drop=True, inplace=True)

    return df

# ************************************************** 
# craetes a step df which is the merge between cities - left 
# and main - right; which is specific df for given step
# extended version when we have to add ratio parameters to value
# **************************************************
def init_step_df_ext(cities_df, main_df, param, ratio_df, ratio):
    ratio_param = '_'.join([ratio, param, 'drop'])
    rate_ratio = '_'.join(['rate', ratio, 'drop'])
    parent_param = '_'.join(['parent', param, 'drop'])
    unit_param = '_'.join(['unit', param, 'drop'])
    
    step_df = init_step_df(cities_df, main_df)
    step_df = enrich_with_parent(step_df, main_df, param) # adds data for parent
    step_df = enrich_with_ratio(step_df, ratio_df, ratio) # adds ratio rate
    
    step_df[ratio_param] = step_df[rate_ratio] * step_df[parent_param]
    step_df[param] = np.where(step_df[unit_param] == 0, step_df[ratio_param], step_df[unit_param])
    
    return step_df


# ************************************************** 
# enriches a step df with parent data
# used for employment, unemployed and revenue df
# **************************************************
def enrich_with_parent(step_df, main_df, param):
    drop_param = '_'.join([param, 'drop'])
    parent_param = '_'.join(['parent', param, 'drop'])
    unit_param = '_'.join(['unit', param, 'drop'])
    
    step_df = pd.merge(step_df, main_df[['id', 'year', param]], left_on=['parentId', 'year'], right_on=['id', 'year'], suffixes=('', '_drop'))
    step_df.rename(columns={param: unit_param, drop_param: parent_param}, inplace=True)
    step_df.drop(['id_drop'], axis=1, inplace=True)

    return step_df


# ************************************************** 
# enriches a step df with ratio which can be used to fill statistics gaps - applied to parent data
# used for employment, unemployed and revenue df
# **************************************************
def enrich_with_ratio(step_df, ratio_df, ratio):
    drop_ratio = '_'.join([ratio, 'drop'])
    parent_ratio = '_'.join(['parent', ratio, 'drop'])
    rate_ratio = '_'.join(['rate', ratio, 'drop'])

    step_df = pd.merge(step_df, ratio_df[['id', 'year', ratio]], on=['id', 'year'])
    step_df = pd.merge(step_df, ratio_df[['id', 'year', ratio]], left_on=['parentId', 'year'], right_on=['id', 'year'], suffixes=('', '_drop'))
    step_df.rename(columns={drop_ratio: parent_ratio}, inplace=True)
    step_df.drop(['id_drop'], axis=1, inplace=True)
    step_df[rate_ratio] = step_df[ratio] / step_df[parent_ratio]

    return step_df


# ************************************************** 
# drops from the df all columns with 'drop' suffix
# sorts and reindexes
# **************************************************
def cleanup_df(df):
    df.drop([col for col in df.columns if 'drop' in col], axis=1, inplace=True)
    df.sort_values(sort_order, inplace=True)
    df.reset_index(drop=True, inplace=True)


# ************************************************** 
# craetes a stats df based on step df
# we filter out data for the end of given periods
# we drop param column as we are not interested in value for current year
# we are more interested in aggregation in the 5 year span 
# **************************************************
def init_stats_df(step_df, drop_list):
    df = step_df[step_df['year'].isin([x + period_length for x in periods])].copy()
    df = df.dropna()
    df.drop(drop_list, axis=1, inplace=True)
    df.reset_index(drop=True, inplace=True)

    return df


# ************************************************** 
# calculates a quantile score for given stats
# **************************************************
def stats_quantile_score(stats_df, score, reversed=False):
    stats_df[score] = np.NaN

    by_year = stats_df.groupby('year')
    for year, frame in by_year:
        quantiles = frame['mean'].quantile(quantile_distribution).values.tolist()
        frame[score] = frame.apply(lambda row: quantile_score(row['mean'], quantiles, reversed=reversed), axis=1)
        stats_df.update(frame)

# ************************************************** 
# appends stats dataframe to existing one for given param
# **************************************************
def append_stats(stats_df, append_df, param): 
    param_score = '_'.join([param, 'score'])
    param_mean = '_'.join([param, 'mean'])
    
    df = pd.merge(stats_df, append_df[['id', 'year', 'mean', param_score]], left_on=['id', 'year'], right_on=['id', 'year'])
    df.rename(columns={'mean': param_mean}, inplace=True)

    df['score'] = df['score'] + df[param_score]

    return df

# Data

## Main Dataframes

In [7]:
units_df = pd.read_csv('../data/bdl/units.csv', dtype={'id': str, 'parentId': str})
population_df = init_main_df('../data/bdl/data_unit_population_short.csv', 72305, 'population')
migration_df = init_main_df('../data/bdl/data_unit_migration_outer_short.csv', 1365234, 'migration')
working_age_df = init_main_df('../data/bdl/data_unit_working_age_short.csv', 152, 'working_age')
employment_df = init_main_df('../data/bdl/data_unit_employment_short.csv', 54821, 'employment')
unemployed_df = init_main_df('../data/bdl/data_unit_registered_unemployed_short.csv', 10514, 'unemployed')
own_revenue_df = init_main_df('../data/bdl/data_unit_own_revenue_short.csv', 76070, 'own_revenue')

units_gcs_df = pd.read_csv('../data/arcgis/units_gcs.csv', dtype={'id': str})


## Cities

In [8]:
community_lvl = [6]
city_types = [1, 4]

cities_df = units_df[(units_df['level'].isin(community_lvl)) & units_df['kind'].isin(city_types)].copy()

cities_df = cities_df[~cities_df['id'].isin(logical_units)]
cities_df = cities_df[~cities_df['id'].isin(warsaw_districts)]
cities_df = cities_df[~cities_df['id'].isin(merge_dict.keys())]

# below could be implemented as lambda but we would have to move column to position 1
teryt_id = cities_df['id'].str[2:4] + cities_df['id'].str[7:12]
cities_df.insert(1, 'teryt_id', teryt_id)

cities_df['name'] = cities_df.apply(lambda row: city_name(row['name']), axis=1)

# clean up
cities_df.drop(['level', 'kind', 'hasDescription', 'description', 'years'], axis=1, inplace=True)
cities_df.reset_index(drop=True, inplace=True)


# Methodology

## Population

In [9]:
param = 'population'
score = '_'.join([param, 'score'])

step_df = init_step_df(cities_df, population_df)

step_df['diff'], step_df['rate'], step_df['mean'] = np.NaN, np.NaN, np.NaN
by_id = step_df.groupby('id')
for id, frame in by_id:
    frame['diff'] = frame[param].diff()
    frame['rate'] = frame['diff'] / (frame[param] - frame['diff'])
    frame['mean'] = frame['rate'].rolling(window=5).mean()
    step_df.update(frame)

stats_df = init_stats_df(step_df, [param, 'parentId', 'diff', 'rate'])
stats_quantile_score(stats_df, score)

population_step_df = step_df
population_step_df.to_csv(csv_name('../figures', 'step_population'))
population_stats_df = stats_df
population_stats_df.to_csv(csv_name('../figures', 'stats_population'))


## Migration

In [10]:
param = 'migration'
score = '_'.join([param, 'score'])

step_df = init_step_df(cities_df, migration_df)

step_df = pd.merge(step_df, population_df[['id', 'year', 'population']], on=['id', 'year'])
step_df['rate'] = step_df['migration'] / (0.001 * step_df['population'])

step_df['mean'] = np.NaN
by_id = step_df.groupby('id')
for id, frame in by_id:
    frame['mean'] = frame['rate'].rolling(window=5).mean()
    step_df.update(frame)

stats_df = init_stats_df(step_df, [param, 'parentId', 'population', 'rate'])
stats_quantile_score(stats_df, score)

migration_step_df = step_df
migration_step_df.to_csv(csv_name('../figures', 'step_migration'))
migration_stats_df = stats_df
migration_stats_df.to_csv(csv_name('../figures', 'stats_migration'))


## Working Age

In [11]:
param = 'working_age'
score = '_'.join([param, 'score'])

step_df = init_step_df(cities_df, working_age_df)

step_df['diff'], step_df['rate'], step_df['mean'] = np.NaN, np.NaN, np.NaN
by_id = step_df.groupby('id')
for id, frame in by_id:
    frame['diff'] = frame[param].diff()
    frame['rate'] = frame['diff'] / (frame[param] - frame['diff'])
    frame['mean'] = frame['rate'].rolling(window=5).mean()
    step_df.update(frame)

stats_df = init_stats_df(step_df, [param, 'parentId', 'diff', 'rate'])
stats_quantile_score(stats_df, score)

working_age_step_df = step_df
working_age_step_df.to_csv(csv_name('../figures', 'step_working_age'))
working_age_stats_df = stats_df
working_age_stats_df.to_csv(csv_name('../figures', 'stats_working_age'))

## Employment

In [12]:
param = 'employment'
score = '_'.join([param, 'score'])
ratio = 'working_age'

step_df = init_step_df_ext(cities_df, employment_df, param, working_age_df, ratio)
cleanup_df(step_df)
step_df = step_df.astype({param: int})

step_df['diff'], step_df['rate'], step_df['mean'] = np.NaN, np.NaN, np.NaN
by_id = step_df.groupby('id')
for id, frame in by_id:
    frame['diff'] = frame[param].diff()
    frame['rate'] = frame['diff'] / (frame[param] - frame['diff'])
    frame['mean'] = frame['rate'].rolling(window=5).mean()
    step_df.update(frame)

stats_df = init_stats_df(step_df, [param, ratio, 'parentId', 'diff', 'rate'])
stats_quantile_score(stats_df, score)

employment_step_df = step_df
employment_step_df.to_csv(csv_name('../figures', 'step_employment'))
employment_stats_df = stats_df
employment_stats_df.to_csv(csv_name('../figures', 'stats_employment'))


## Registered Unemployed

In [13]:
param = 'unemployed'
rate = '_'.join([param, 'rate'])
score = '_'.join([param, 'score'])
ratio = 'working_age'

step_df = init_step_df_ext(cities_df, unemployed_df, param, working_age_df, ratio)
cleanup_df(step_df)
step_df = step_df.astype({param: int})

# first we need to calculate a unemployed_rate for that we need value of working_age population
# it just happened that working age and ratio are the same
# we count unemployed rate in % points, that's why we don't have ratio year by year
step_df[rate], step_df['diff'], step_df['mean'] = np.NaN, np.NaN, np.NaN
by_id = step_df.groupby('id')
for id, frame in by_id:
    frame[rate] = frame[param] / frame[ratio]
    frame['diff'] = frame[rate].diff()
    frame['mean'] = frame['diff'].rolling(window=5).mean()
    step_df.update(frame)

stats_df = init_stats_df(step_df, [param, ratio, 'parentId', 'diff'])
stats_quantile_score(stats_df, score, reversed=True)

unemployed_step_df = step_df
unemployed_step_df.to_csv(csv_name('../figures', 'step_unemployed'))
unemployed_stats_df = stats_df
unemployed_stats_df.to_csv(csv_name('../figures', 'stats_unemployed'))


## Own Revenue

In [14]:
param = 'own_revenue'
param_pp = '_'.join([param, 'pp'])
score = '_'.join([param, 'score'])
ratio = 'working_age'

step_df = init_step_df_ext(cities_df, own_revenue_df, param, working_age_df, ratio)
cleanup_df(step_df)

step_df = pd.merge(step_df, population_df[['id', 'year', 'population']], on=['id', 'year'])
step_df[param_pp] = step_df['own_revenue'] / step_df['population']

step_df['diff'], step_df['rate'], step_df['mean'] = np.NaN, np.NaN, np.NaN
by_id = step_df.groupby('id')
for id, frame in by_id:
    frame['diff'] = frame['own_revenue_pp'].diff()
    frame['rate'] = frame['diff'] / (frame[param_pp] - frame['diff'])
    frame['mean'] = frame['rate'].rolling(window=5).mean()
    step_df.update(frame)

stats_df = init_stats_df(step_df, [param, param_pp, ratio, 'parentId', 'population', 'diff', 'rate'])
stats_quantile_score(stats_df, score)

own_revenue_step_df = step_df
own_revenue_step_df.to_csv(csv_name('../figures', 'step_own_revenue'))
own_revenue_stats_df = stats_df
own_revenue_stats_df.to_csv(csv_name('../figures', 'stats_own_revenue'))

## Statistics

In [15]:
stats_df = population_stats_df[['id', 'teryt_id', 'name', 'year']].copy()
stats_df = pd.merge(stats_df, population_df[['id', 'year', 'population']], left_on=['id', 'year'], right_on=['id', 'year'])

# applying city type could be also done in a dashboard esp when we have a ratio parameter included
stats_df['city_type'] = stats_df.apply(lambda row: city_type(row['population']), axis=1)
stats_df = pd.merge(stats_df, units_gcs_df[['id', 'latitude', 'longitude']], on=['id'])
stats_df['score'] = 0
stats_df['status'] = np.NaN

stats_df = append_stats(stats_df, population_stats_df, 'population')
stats_df = append_stats(stats_df, migration_stats_df, 'migration')
stats_df = append_stats(stats_df, working_age_stats_df, 'working_age')
stats_df = append_stats(stats_df, employment_stats_df, 'employment')
stats_df = append_stats(stats_df, unemployed_stats_df, 'unemployed')
stats_df = append_stats(stats_df, own_revenue_stats_df, 'own_revenue')

stats_df['status'] = stats_df.apply(lambda row: city_status(row['score']), axis=1)

by_year = stats_df.groupby('year')
for year, frame in by_year:    
    name = '_'.join([str(int(year)), 'shrinking_cities'])
    frame.to_csv(csv_name('../figures', name))


# Archive

In [16]:
# Below code is an example how to find list of unit ids which require to be unified

# df = units_df[units_df['description'].str.len() > 0] # 675
# df = df[~df['description'].str.startswith('Zmiana rodzaju gminy z wiejskiego na miejsko-wiejski')] # 675 - 406 = 269
# df = df[~df['description'].str.startswith('Zmiana granic')] # 269 - 25 = 244
# df = df[~df['description'].str.startswith('Utworzenie')] # 244 - 32 = 212
# df = df[~df['description'].str.startswith('Zmiana przynależności')] # 212 - 114 = 98 ***
# df = df[~df['description'].str.startswith('Zmiana rodzaju gminy z miejskiego na miejsko-wiejski')] # 98 - 20 = 78 ***
# df = df[~df['description'].str.startswith('Przeniesienie gminy')] # 78 - 4 = 74 ***
# df = df[~df['description'].str.startswith('Zmiana symbolu gminy')] # 74 - 2 = 72 ***
# df = df[~df['description'].str.startswith('Zmiana rodzaju gminy')] # 72 - 6 = 66 *** / leftovers 
# df = df[~df['description'].str.startswith('Zmiana ustroju miasta')] # 66 - 2 = 64 ***
# df = df[~df['description'].str.startswith('Przywrócenie miastu Wałbrzych statusu')] # 64 - 2 = 62 ***
# df = df[~df['description'].str.startswith('Zniesienia gminy')] # 62 - 1 = 61
# df = df[~df['description'].str.startswith('Połączenie miasta')] # 61 - 1 = 60
# df = df[~df['description'].str.startswith('Zniesienie')] # 60 - 18 = 42 ***
# df = df[~df['description'].str.startswith('Zmiana nazwy')] # 42